In [ ]:
from functools import partial

import numpy as np
import tensorflow as tf
import nbimporter 
from optimizers import create_train_op
from metric_fns import *

In [ ]:
# Xây dựng    model GPT 2 
def gpt2_model(features, labels, mode , params):
    # import GPT2
    from models.gpt2 import gpt2

    # kiểm tra xem model có phải là đang ở chế độ train hoặc validation không 
    if mode == tf.estimator.ModeKeys.Train or mode == tf.estimator.ModeKeys.EVAL:
        # KIỂM tra xem giá trị khóa precision trong từ điển params có phải là kiểu bfloat16
        if params['percision'] == 'bfloat16':
            # khởi tạo 1 biến phạm vi bfloat16 
            with tf.contrib.tpu.bfloat16_scope():
                # Train or eval model và gán kết quả cho biến output 
                output = gpt2.model(X=features, params=params, labels=labels,
                                past = None, reuse=tf.AUTO_REUSE, 
                                train=mode==tf.estimator.ModeKeys.TRAIN)
                
            # Gán giá trị của logits trong danh sách output = kiểu float32
            output["logits"] = tf.cast(output["logits"], tf.float32)

        else : # Trường hợp còn lại ta thực hiện tương tự
            output = gpt2.model(X=features, params=params,
                                    labels=labels,
                                    past=None, reuse=tf.AUTO_REUSE,
                                    train=mode==tf.estimator.ModeKeys.TRAIN)
        # Tính loss sử dụng hàm sparse_softmax_cross_entropy_with_logits
        # với danh ách tham số logits
        loss_batch = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output["logits"], labels=labels)
        # Gán loss = kết quả cuối cùng 
        loss = tf.reduce_mean(loss_batch)
    
    # Kiểm tra xem mode có = chế độ train 
    if mode == tf.estimator.ModeKeys.TRAIN:
        # Tối ưu hóa tham số llosss 
        train_op = create_train_op(loss, params)
        
        # kiểm tra xem giá trị của khóa Use_tpu = True tức đang sử dụng TPU 
        if params["use_tpu"]:
            # sử dụng  tf.contrib.tpu.TPUEstimatorSpec dùng để cấu hình cho tham số mode
            # (Huấn luyện và đánh gía trên TPU) và nhúng đánh giá tương tự như EstimatorSpec
            # nhưng được tối ưu hóa cho TPU 
            return tf.contrib.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)
        
        # Trường hợp không có sử dụng tpu 
        else: 
            # Sử dụng tf.estimator.EstimatorSpace để cấu hình cho tham số huấn luyện hoặc đánh giá mô hình
            # và nhúng đánh giá tham số 
            return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
        
    # nếu mô hình đang ở chế độ đào tạo Mode = Eval 
    if mode == tf.estimator.ModeKeys.EVAL: 
        # Import phuuwong thức perplexity_metric để tính loss 
        from metric_fns import perplexity_metric 

        # kiểm tra xem mô hình có đang sử dụng TPU hay không 
        if params["use_tpu"]:
            # Metric inputs are transferred to CPU and must preserve batch dimension
            # Đầu vào số liệu được chuyển đến CPU và phải bảo toàn kích thước lô 

            # Sử dụng tf.contri.tpu.TPUEstimatorSpace dùng để cấu hình cho tham số của Eval 
            # và nhúng đánh giá 
            return tf.contrib.tpu.TPUEstimatorSpace(
                # Tính loss và áp dụng phương thức perplexity đê tính toán loss cho trình Eval 
                # các giá trị này sẽ tạo ra loss_batch và lưu trữ dạng từ điển 
                loss=loss , mode=mode , eval_metrics=(perplexity_metric, {"loss": loss_batch})
            )
        
        # Trường hợp còn lại ta thực hiện tương tự như với mode Train 
        else: 
            # sử dụng tf.estiamtor.EstimatorSpec dùng để cấu hình cho tham số EVAL 
            # và nhúng đánh gia tham số loss 
            return tf.estimator.EstimatorSpec(mode=mode,
                loss=loss, eval_metric_ops=perplexity_metric(loss_batch))

    # Trường hợp nếu như mô hình đang ở chế độ dự đoán 
    if mode == tf.estimator.ModeKeys.PREDICT:
        # import phương thức sample từ model GPT2 để thực hiện lấy 
        # mẫu và tính xác suất cho đầu ra 
        from model.gpt2 import sample 

        # kiển tra xem trong từ điển params có tồn tại top_k 
        if not "top_k" in params.keys():
            # gám cho giá trị key (top_k) trong từ điển params = 0
            params["top_k"] = 0
        
        # Thực hiện lấy mẫu trình tự và gán nó cho output 
        output = sample.sample_sequence(
            params=params , length=params['n_ctx'],
            context = features, 
            batch_size = params['batch_size'], 
            temperature=1.0, top_k=params['top_k']
        )
        # tạo 1 từ điển predict keys = tokens và values = ouput
        predictions = {
            "tokens": output
        }
        # kiểm tra xen chế độ này có đáng ử dụng tpu
        if params["use_tpu"]:
            # sử dụng tf.contrib.tpu.TPUEstimatorSPace dùng để cấu hình các tham số 
            # cho chế độ Predict (trường hợp này không sử dụng nhúng đánh giá như 2 trường hợp trên)
            return tf.contrib.tpu.TPUEstimatorSpec(mode , predictions=predictions)

        else:
            # # sử dụng tf.estiamtor.EstimatorSpec dùng để cấu hình cho tham số Preidict 
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)
            
    